------------------------------------------------------------------------------------------------------------------------------------------------------

## CROSSTALK NOTEBOOK

------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
import sys; sys.path.insert(0, '../'); from lib import *;

In [ ]:
# Set options for general visualitation
OPT  = {
    "MICRO_SEC":   True,                # Time in microseconds (True/False)
    "NORM":        False,               # Runs can be displayed normalised (True/False)
    "ALIGN":       True,                # Aligns waveforms in peaktime (True/False)
    "LOGY":        False,               # Runs can be displayed in logy (True/False)
    "SHOW_AVE":    "",                  # If computed, vis will show average (AveWvf,AveWvfSPE,etc.)
    "SHOW_PARAM":  False,               # Print terminal information (True/False)
    "CHARGE_KEY":  "ChargeAveRange",    # Select charge info to be displayed. Default: "ChargeAveRange" (if computed)
    "PEAK_FINDER": False,               # Finds possible peaks in the window (True/False)
    "LEGEND":      True,                # Shows plot legend (True/False)
    "STYLE":       "CIEMAT_style",      # Plot style. Default: "CIEMAT_style"
    "SHOW":        True
    }

style_selector(OPT)

The following cell can be used to load the configuration file.
Check that everything is OK and choose your runs!

In [ ]:
info = read_input_file("TUTORIAL")     # Read input file
for key in info: print(key,info[key])  # Print loaded info
for key in info: print(key,info[key])  # Print loaded info

In [ ]:
chs  = [0,6]
runs = [1] 
my_runs = load_npy(np.asarray(runs).astype(int),np.asarray(chs).astype(int),preset="EVA",info=info,compressed=True) # LOAD YOUR RUNS

# Vinogradov model (Poisson+Binomial distribution)

In [ ]:
def fit_gaussians(x, y, *p0):
    assert x.shape == y.shape, "Input arrays must have the same shape."
    popt, pcov = curve_fit(gaussian_train, x,y, p0=p0[0])
    fit_y=gaussian_train(x,*popt)
    chi_squared = np.sum((y[abs(fit_y)>0.1] - fit_y[abs(fit_y)>0.1]) ** 2 / fit_y[abs(fit_y)>0.1]) / (y.size - len(popt))
    return popt,fit_y, chi_squared


In [ ]:
# file_CT=[];
debug = True
factor2PE = 1/3.48e2 # GAIN IN ADC*ticks TO CONVER CHARGE INTO PE
my_bins   = 350
my_range  = [-1,10]
ch  = 0
run = 1

#get counts
counts,bins= np.histogram(my_runs[run][ch]["AnaChargePedRange1"]*factor2PE,my_bins,my_range);#need to convert to PE

#find peaks
peaks = find_peaks(counts,height=150,width=4,distance=int(my_bins/((my_range[1]-my_range[0])*1.2))  )
print("FOUNDED PEAKS:", peaks)

# right limit:(supossing same space between peaks, prevents adding more gaussians that the ones considered to be fitted)
r_lim = peaks[0][-1]+ int (((peaks[0][-1]-peaks[0][-2]))/2)
l_lim = peaks[0][ 0]- int (((peaks[0][ 1]-peaks[0][ 0]))/2)

params       = np.zeros(len(peaks[0])*3)
params[0::3] = peaks[1]["peak_heights"]
params[1::3] = bins[peaks[0]]
params[2::3] = 0.5
my_vars,fit_y,qs = fit_gaussians(bins[:-1][l_lim:r_lim],counts[l_lim:r_lim],params)

# 1st Plot: gaussian fits
plt.figure(figsize=(5,3)) #dpi=200
plt.hist(my_runs[run][ch]["AnaChargePedRange1"]*factor2PE,bins=my_bins,range=my_range,histtype="step",label="Data",linewidth=1.5)
plt.xlim([-1,7]) 
plt.plot(bins[peaks[0]],peaks[1]["peak_heights"],"x",label="Peaks")
plt.plot(bins[:-1][l_lim:r_lim] +(bins[1]-bins[0])/2  ,fit_y,'--',color="red",label="Best fit")        
plt.legend(fontsize=10)
plt.xlabel(my_runs[run][ch]["Label"] +" Charge [PEs]")
plt.ylabel("Counts")
plt.grid()

Amp=my_vars[0::3]; sigma=np.abs(my_vars[2::3]) #Compute Gaussian areas
PNs=Amp*sigma/sum(Amp*sigma) #prob is proportional to A*sigma (sqrt(2pi))

PNs_err=(Amp*sigma)**0.5/sum(Amp*sigma) #assuming error propt to sqrt(N)
if debug:
    print("Amplitude:" ,Amp)
    print("Sigma: ",sigma)
    print("PNs: ", PNs)
    print("PNs_error: ",PNs_err)

P0 = PNs[0]
P1 = PNs[1]
l  = -np.log(P0)
p  = 1-P1/(l*P0)
print("Initial vars:",p,"\t",l)

# 2nd plot, Vinogradov fit
fig, ax = plt.subplots(1,figsize=(5,3)) #dpi=200,
plt.xlim([-.5,5.5]) 
# Get the peaks that were fitted
a     = np.arange(my_bins)*(my_range[1]-my_range[0])/my_bins
b     = a[peaks[0]]-1
xdata = b.round().astype(int).tolist()
if debug: print(b,xdata)


plt.errorbar(x=xdata,y=PNs,yerr=PNs_err, color="k",linestyle="none",marker="s",markersize=2,capsize=2,)
plt.bar(np.array(xdata),PNs,label="Data",width=0.4)

p0 = [p,l]
popt, pcov = curve_fit(PoissonPlusBinomial, xdata,PNs,sigma=PNs_err, p0=p0)
plt.plot(xdata, PoissonPlusBinomial(xdata, *popt), 'x',label="Fit: CT=" +str(int(popt[0]*100)) +"%",color="red")

plt.ylabel("Counts [Density]")
plt.xlabel("N. of photoelectrons")
plt.legend(fontsize=10)
# Display also the ideal poisson dist
ideal=poisson.pmf(xdata, mu=popt[1])
plt.bar(np.array(xdata)+.3,ideal,width=.3)

print("Fitted vars: ",popt)
perr = np.sqrt(np.diag(pcov))
print("Rel Error: ", perr/popt*100)

#save values to file
# file_CT.append([OV,popt[0],perr[0]])
# arr = np.asarray(file_CT)
# np.savetxt('CT_'+WEEK+"_CH_"+Calib_run["ChannelName"][Chan_dic[ch]]+ '_.csv',arr, delimiter=",")  

## Demostration plot: poisson vs poisson+binomial

In [ ]:
L=5
x=poisson.pmf(range(20), mu=L)

# plt.figure(dpi=200)
plt.plot(x,label="Pure Poisson dist")
for ct in np.arange(0.05,.4,0.1):
    # x=[F(i,ct,L) for i in range(20)]
    x=PoissonPlusBinomial(range(20), ct, L)
    plt.plot(x,'o-',label="Poisson +  Binomial p*100="+str(int(ct*100)) )

plt.legend()
plt.show()